In [6]:
using Revise, Statistics, LaTeXStrings, DSP, StatsBase
using Formatting
using CairoMakie
using ColorSchemes
import ColorSchemes: nord, tab10, Dark2_8

printx(x,N) = sprintf1("%10.$(N)f",x)


# fg = :black  # nord[5]
# gg = :grey   # nord[3]
fg = nord[5]
gg = nord[3]
bg = :transparent

set_theme!(Theme(
    fontsize = 20,
    palette = ( color = tab10,),
    Axis = (
        backgroundcolor = bg,
        xtickcolor = fg, ytickcolor = fg,
        xgridcolor = gg, ygridcolor = gg,
        xlabelcolor = fg, ylabelcolor = fg,
        xticklabelcolor = fg, yticklabelcolor = fg,
        topspinecolor = fg, bottomspinecolor = fg,
        leftspinecolor = fg, rightspinecolor = fg,
        titlecolor = fg,
    ),
    Legend = (
        bgcolor = bg,
        labelcolor = fg, titlecolor = fg,
        framevisible = false, margin=(0,0,0,0)
    )
))

In [179]:
# function covpred(x, p, r)
#     N = length(x)
#     # Simply write the definition of the autocorrelationfunction
#     acf(k) = sum([x[n+1]*x[n-k+1] for n=k:N-1])
#     # Use ACF to find the covariance matrix
#     R = [ acf(abs(i-j)) for i=0:p, j=0:p ]
#     a = R \ [ x[1]; zeros(p)]
#     err = mean((x - filt(1.0, a, [1; zeros(N-1)])).^2)
#     return [a, err]
# end

function covpred(x, p, r)
    N = length(x)
    X = zeros(N+p, p+1)
    for i=1:p+1
        X[i:i+N-1,i] = x
    end
    b = [x[1]; zeros(N+p-1)]
    a = X \ b
    err = mean((x - filt(1.0, a, [1; zeros(N-1)])).^2)
    return [a, err]
end

function corrpred(x, p, r)
    N = length(x)
    X = zeros(N+p, p+1)
    p = p-1
    for i=1:p+1
        X[i:i+N-1,i] = x
    end
    X = X[p+1:N-1,:] 
    b = x[p+2:N]
    a = [ 1; -X \ b ]
    err = mean((x - filt(1.0, a, [1; zeros(N-1)])).^2)
    return [a, err]
end

corrpred (generic function with 1 method)

In [180]:
x = filt(1.0, [1,0.2,0.3], [1; zeros(100)])
@show covpred(x, 2, 0);
@show corrpred(x, 2, 0);

covpred(x, 2, 0) = Any[[1.0, 0.20000000000000004, 0.3], 4.523690588545417e-35]
corrpred(x, 2, 0) = Any[[1.0, 0.19999999999999998, 0.29999999999999993, 0.0], 5.611008013959383e-35]


LoadError: error in method definition: function Base.reduce must be explicitly imported to be extended